In [5]:
import torch
import torch.nn as nn
import numpy as np
import collections
import gym
import ptan
from typing import List, Optional, Tuple, Any

In [5]:
class CustomNN(nn.Module):
    def __init__(self, input_shape, n_actions):
        super(CustomNN, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(input_shape, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, n_actions),
        )

    def forward(self, x):
        return self.net(x)

In [44]:
net = CustomNN(4, 4)
actions_v = torch.tensor(np.array([0, 3, 2, 1]), dtype=torch.int64)

In [51]:
out = net(torch.tensor(np.array([[1, 2, 3, 4], [4, 3, 2, 1]]), dtype=torch.float32))
actions_v.unsqueeze(-1).squeeze(-1)
# out

tensor([0, 3, 2, 1])

In [3]:
class ToyEnv(gym.Env): 
    def __init__(self): 
        super(ToyEnv, self).__init__()
        self.observation_space = gym.spaces.Discrete(n=5) 
        self.action_space = gym.spaces.Discrete(n=3)
        self.step_index = 0 
    
    def reset(self): 
        self.step_index = 0 
        return self.step_index 
        
    def step(self, action): 
        is_done = self.step_index == 10 
        if is_done: 
            return (
                self.step_index % self.observation_space.n, 
                0.0, is_done, 
                {}
                )
        self.step_index += 1 
        return (
            self.step_index % self.observation_space.n, 
            float(action), 
            self.step_index == 10, 
            {}
        )


In [6]:
class DullAgent(ptan.agent.BaseAgent): 
    """ Agent always returns the fixed action """ 
    def __init__(self, action: int): 
        self.action = action 
    
    def __call__(self, observations: List[Any], state: Optional[List] = None) -> Tuple[List[int], Optional[List]]: 
        return [self.action for _ in observations], state


In [31]:
env = ToyEnv()
agent = DullAgent(action=1)
exp_source = ptan.experience.ExperienceSource(
    env=[env, env],
    agent=agent,
    steps_count=8
)

In [32]:
for idx, exp in enumerate(exp_source):
    if idx > 20:
        break

    print(len(exp))

5
4
3
2
1
8
8
8
5
4
3
2
1
8
8
8
8
8
5
4
3
